By: 10117094	Muhammad Walidin
    10116476	Idham Rahadian
    10116490	Muhammad Fajar Maulana

In [1]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter

In [2]:
raw = pd.read_csv( "data/data/1.1_data_review_clean.csv")
slang_words = pd.read_csv("data/external/kbba_ID.txt",
                        sep="\t", header=None)
slang = pd.read_csv("data/external/slangword_ID.txt",
                        sep=":", header=None)
baku_words = pd.read_csv("data/external/katabaku_ID.txt",
                        sep="|", header=None)
baku_words.columns = [1,0]
root_words = np.array(pd.read_csv("data/external/rootword_ID.txt",
                        sep="\n", header=None).values)
slang_words = pd.concat([slang_words, slang, baku_words])

In [3]:
raw

,rating,review,avg_word,word_count
0,5,"'tidak', 'ada', 'ulasan'",7.333333,3
1,5,"'puas', 'kecil', 'tapi', 'kencang'",7.750000,4
2,1,"'barang', 'yang', 'datang', 'bukan', 'baracuda...",7.923077,13
3,5,"'terima kasih', 'barang', 'sudah', 'sampai', '...",9.117647,17
4,5,"'barang', 'bagus', 'pengirman', 'cepat', 'mant...",8.500000,6
...,...,...,...,...
799,5,"'barang', 'kondisi', 'oke', 'respon', 'cepat',...",7.875000,8
800,5,"'barang', 'sampai', 'tepat', 'waktu', 'sesuai'...",8.272727,22
801,5,"'pelayanan', 'sangat', 'bagus', 'barang', 'ses...",8.650000,20
802,5,"'barang', 'sampai', 'lebih', 'cepat', 'dan', '...",7.833333,18


In [4]:
slang_words.drop_duplicates(inplace=True)
slang_words = dict(zip(slang_words[0],slang_words[1]))
raw['review'] = raw['review'].apply(lambda x: x.replace(","," "))
def delete_suffix_nya(review):
    return re.sub("(?:nya|ny|y)[$|\s]"," ",review)
    
raw['review'] = raw['review'].apply(delete_suffix_nya)
raw['review'] = raw['review'].apply(lambda x: x.split())

In [5]:
raw

,rating,review,avg_word,word_count
0,5,"['tidak', 'ada', 'ulasan']",7.333333,3
1,5,"['puas', 'kecil', 'tapi', 'kencang']",7.750000,4
2,1,"['barang', 'yang', 'datang', 'bukan', 'baracud...",7.923077,13
3,5,"['terima, kasih', 'barang', 'sudah', 'sampai',...",9.117647,17
4,5,"['barang', 'bagus', 'pengirman', 'cepat', 'man...",8.500000,6
...,...,...,...,...
799,5,"['barang', 'kondisi', 'oke', 'respon', 'cepat'...",7.875000,8
800,5,"['barang', 'sampai', 'tepat', 'waktu', 'sesuai...",8.272727,22
801,5,"['pelayanan', 'sangat', 'bagus', 'barang', 'se...",8.650000,20
802,5,"['barang', 'sampai', 'lebih', 'cepat', 'dan', ...",7.833333,18


In [6]:
def mapping_slang_words(review):
    return [slang_words[word] if word in slang_words else word for word in review]
raw['review'] = raw['review'].apply(mapping_slang_words)

In [7]:
def convert_list_to_string(word_list):
    return ",".join(word_list)
raw['review'] = raw['review'].apply(convert_list_to_string)
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]

In [8]:
def avg_word(words):
  return (sum(len(word) for word in words)/len(words))

raw['avg_word'] = raw['review'].apply(lambda x: avg_word(x.split(",")))

In [9]:
raw.to_csv("data/data/1.2_data_review_cleaner_pure.csv", index=False)

In [10]:

stop_words = np.array(pd.read_csv("data/external/stopwords_ID.txt",
                        sep="\n", header=None).values)
neg_words = np.array(pd.read_csv("data/external/negative_keyword_ID.txt",
                        sep="\n", header=None).values)
pos_words = np.array(pd.read_csv("data/external/positive_keyword_ID.txt",
                        sep="\n", header=None).values)
booster_words = np.array(pd.read_csv("data/external/boosterword_ID.txt",
                        sep="\n", header=None).values)
sentiment_words = np.concatenate((pos_words, neg_words, booster_words))
stop_words = np.reshape(stop_words, -1)

In [11]:
raw.head()
raw['review'] = raw['review'].apply(lambda x: x.split(","))

In [12]:
raw.head()

,rating,review,avg_word,word_count
0,5,"['tidak', 'ada', 'ulasan']",6.666667,3
1,5,"['puas', 'kecil', 'tapi', 'kencang']",7.000000,4
2,1,"['barang', 'yang', 'datang', 'bukan', 'baracud...",6.428571,13
3,5,"['terima, kasih', 'barang', 'sudah', 'sampai',...",7.666667,17
4,5,"['barang', 'bagus', 'pengirman', 'cepat', 'man...",7.666667,6


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
tvec_weights = vectorizer.fit_transform(raw['review'])
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'weight': weights})
weights_df = weights_df.sort_values(by='weight', ascending=False)
domain_spesific_stop_words = np.array(weights_df['term'][:200].values)
stop_words = np.concatenate((domain_spesific_stop_words,stop_words[:,]))
stop_words = [word for word in stop_words if word not in sentiment_words]
stop_words = np.unique(stop_words)

In [14]:
raw.head()

,rating,review,avg_word,word_count
0,5,"['tidak', 'ada', 'ulasan']",6.666667,3
1,5,"['puas', 'kecil', 'tapi', 'kencang']",7.000000,4
2,1,"['barang', 'yang', 'datang', 'bukan', 'baracud...",6.428571,13
3,5,"['terima, kasih', 'barang', 'sudah', 'sampai',...",7.666667,17
4,5,"['barang', 'bagus', 'pengirman', 'cepat', 'man...",7.666667,6


In [15]:
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]
def avg_word(words):
  return (sum(len(word) for word in words)/len(words))

raw['avg_word'] = raw['review'].apply(lambda x: avg_word(x))
raw['word_count'] = raw['review'].map(len)
raw['review'] = raw['review'].apply(convert_list_to_string)

In [16]:
raw.head()

,rating,review,avg_word,word_count
0,5,"'tidak','ada','ulasan'",6.666667,3
1,5,"'puas','kecil','tapi','kencang'",7.000000,4
2,1,"'barang','yang','datang','bukan','baracuda','t...",6.428571,14
3,5,"'terima,kasih','barang','sudah','sampai','seha...",7.666667,18
4,5,"'barang','bagus','pengirman','cepat','mantap',...",7.666667,6


In [17]:
raw.to_csv("data/data/1.2_data_review_cleaner_2.csv", index=False)